In [1]:
from pyspark.sql import SparkSession



from lib.logger import Log4J

if __name__ == "__main__":
    spark = SparkSession \
        .builder \
        .appName("Grouping Demo") \
        .master("local[2]") \
        .getOrCreate()

In [2]:
logger = Log4J(spark)

orders_list = [("01", "02", 350, 1),
                   ("01", "04", 580, 1),
                   ("01", "07", 320, 2),
                   ("02", "03", 450, 1),
                   ("02", "06", 220, 1),
                   ("03", "01", 195, 1),
                   ("04", "09", 270, 3),
                   ("04", "08", 410, 2),
                   ("05", "02", 350, 1)]


In [3]:
order_df = spark.createDataFrame(orders_list).toDF("order_id", "prod_id", "unit_price", "qty")

In [6]:
order_df.show()

+--------+-------+----------+---+
|order_id|prod_id|unit_price|qty|
+--------+-------+----------+---+
|      01|     02|       350|  1|
|      01|     04|       580|  1|
|      01|     07|       320|  2|
|      02|     03|       450|  1|
|      02|     06|       220|  1|
|      03|     01|       195|  1|
|      04|     09|       270|  3|
|      04|     08|       410|  2|
|      05|     02|       350|  1|
+--------+-------+----------+---+



In [4]:
product_list = [("01", "Scroll Mouse", 250, 20),
                    ("02", "Optical Mouse", 350, 20),
                    ("03", "Wireless Mouse", 450, 50),
                    ("04", "Wireless Keyboard", 580, 50),
                    ("05", "Standard Keyboard", 360, 10),
                    ("06", "16 GB Flash Storage", 240, 100),
                    ("07", "32 GB Flash Storage", 320, 50),
                    ("08", "64 GB Flash Storage", 430, 25)]

product_df = spark.createDataFrame(product_list).toDF("prod_id", "prod_name", "list_price", "qty")

In [5]:
product_df.show()


+-------+-------------------+----------+---+
|prod_id|          prod_name|list_price|qty|
+-------+-------------------+----------+---+
|     01|       Scroll Mouse|       250| 20|
|     02|      Optical Mouse|       350| 20|
|     03|     Wireless Mouse|       450| 50|
|     04|  Wireless Keyboard|       580| 50|
|     05|  Standard Keyboard|       360| 10|
|     06|16 GB Flash Storage|       240|100|
|     07|32 GB Flash Storage|       320| 50|
|     08|64 GB Flash Storage|       430| 25|
+-------+-------------------+----------+---+



In [ ]:
join_expr = order_df.prod_id == product_df.prod_id

'''
    order_df.join(product_df, join_expr,"inner") \
        .select("*") \
        .show()
'''

'\n    order_df.join(product_df, join_expr,"inner")         .select("*")         .show()\n'

In [8]:

product_renamed_df = product_df.withColumnRenamed("qty", "reorder_qty")

In [9]:
order_df.join(product_renamed_df, join_expr, "inner") \
        .drop(product_renamed_df.prod_id)\
        .select("order_id", "prod_id","prod_name", "unit_price", "qty") \
        .show()

+--------+-------+-------------------+----------+---+
|order_id|prod_id|          prod_name|unit_price|qty|
+--------+-------+-------------------+----------+---+
|      03|     01|       Scroll Mouse|       195|  1|
|      01|     02|      Optical Mouse|       350|  1|
|      05|     02|      Optical Mouse|       350|  1|
|      02|     03|     Wireless Mouse|       450|  1|
|      01|     04|  Wireless Keyboard|       580|  1|
|      02|     06|16 GB Flash Storage|       220|  1|
|      01|     07|32 GB Flash Storage|       320|  2|
|      04|     08|64 GB Flash Storage|       410|  2|
+--------+-------+-------------------+----------+---+

